In [65]:
# # Use this the first time setting up the conda environment
# !pip install mysql-connector-python
# !pip install pymysql
# !pip install ipython-sql pymysql
# !pip install cryptography

import pymysql
import pandas as pd
import hashlib
import csv

# Base functions referenced in future cells
def hash_password(password): # kayla update this with your preferred hash
    return hashlib.sha256(password.encode()).hexdigest()

In [66]:
### Fill in your user data
user_info = {
    'username': 'rysch01',
    'password': 'SQLP4ssword1!'
}

In [67]:
# Run this cell and it will either connect to your existing database or create it from the sample data
# Connect to the MySQL serve
db = pymysql.connect(
    host='localhost',    
    user=user_info['username'],
    password=user_info['password'],
)

cursor = db.cursor()

try: # try to use database; if so, it's already loaded
    cursor.execute("USE cafeteria")
except: # if not, need to load it in next
    cursor.execute('CREATE DATABASE cafeteria;')
    cursor.execute("USE cafeteria")
    ### Define the students table
    declare_students='''
    CREATE TABLE Students (
        pid INT PRIMARY KEY,
        first_name VARCHAR(255) NOT NULL,
        last_name VARCHAR(255) NOT NULL,
        meal_balance DECIMAL(8, 2) DEFAULT 0.00,
        to_go_boxes_remaining INT DEFAULT 2,
        password_hash CHAR(64) NOT NULL,
        role CHAR(8) DEFAULT "student"
    );
    '''

    ### Define the menu table
    declare_menu='''
    CREATE TABLE Menu (
        item_id INT PRIMARY KEY AUTO_INCREMENT,
        item_name VARCHAR(255) NOT NULL,
        calories_per_serving INT NOT NULL,
        meal_type CHAR(64) NOT NULL,
        available_date DATE NOT NULL,
        price DECIMAL(8, 2) NOT NULL
    );
    '''

    ### Define the transactions table
    declare_transactions='''
    CREATE TABLE Transactions (
        transaction_id INT PRIMARY KEY AUTO_INCREMENT,
        pid INT NOT NULL,
        item_id INT NOT NULL,
        transaction_type VARCHAR(50) NOT NULL,
        transaction_date DATE NOT NULL,
        FOREIGN KEY (pid) REFERENCES Students(pid),
        FOREIGN KEY (item_id) REFERENCES Menu(item_id)
    );
    '''

    # execute and commit changes
    cursor.execute(declare_students)
    cursor.execute(declare_menu)
    cursor.execute(declare_transactions)
    db.commit()

    # load from csv like in HW2

    # Do so for each of the tables
    with open('data/students.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            r[-1]=hash_password(r[-1]) # hash password to store
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Students ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s, %s, %s)', r)

    
    with open('data/menu.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Menu ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s, %s)', r)

    with open('data/transactions.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Transactions ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s)', r)
    # Commit changes
    db.commit()

In [58]:
# ## Insert Query Example
new_item_details = (5, 'Omelette', 600, 'breakfast', '2024-11-12', 8.00)
cursor.execute('INSERT INTO Menu (item_id, item_name, calories_per_serving, meal_type, available_date, price) VALUES (%s, %s, %s, %s, %s, %s);', new_item_details)
db.commit()

# ## Select Query Example
cursor.execute('SELECT * FROM Menu')
result = cursor.fetchall()
print(result)

((1, 'Pasta Primavera', 450, 'lunch', datetime.date(2024, 11, 12), Decimal('11.00')), (2, 'Grilled Chicken Sandwich', 600, 'dinner', datetime.date(2024, 11, 12), Decimal('8.00')), (3, 'Oatmeal', 150, 'breakfast', datetime.date(2024, 11, 12), Decimal('5.00')), (4, 'Vegan Burrito Bowl', 500, 'lunch', datetime.date(2024, 11, 13), Decimal('9.00')), (5, 'Omelette', 600, 'breakfast', datetime.date(2024, 11, 12), Decimal('8.00')))


In [70]:
### Sign-In Page Backend Code + Queries

test_case='admin'

if test_case=='student':
    entered_pid='730330697'
    entered_password='ramses123'
elif test_case=='admin':
    entered_pid='987654321'
    entered_password='flav0rTown!!!'
elif test_case=='wrong_password':
    entered_pid='987654321'
    entered_password='wrong_pass'
elif test_case=='nonexistant_user':
    entered_pid='222222222'
    entered_password='wrongPass'

cursor.execute('SELECT * FROM Students S WHERE S.pid=%s', entered_pid)
result = cursor.fetchall()

entered_password_hash=hash_password(entered_password)
try:
    if (result[0][5]==entered_password_hash):
        print('Sign-In Success')
        current_user = {
            'pid': result[0][0],
            'first_name': result[0][1],
            'last_name': result[0][2],
            'balance': result[0][3],
            'to_go_remaining': result[0][4],
            'password_hash': result[0][5],
            'role': result[0][6],
        }
        print(current_user)

        if current_user['role']=='admin':
            print('direct to admin page')
        else:
            print('direct to student')
    else:
        print('incorrect password code')
except:
    print('pid not found')


Sign-In Success
{'pid': 987654321, 'first_name': 'Guy', 'last_name': 'Feeyarey', 'balance': Decimal('0.00'), 'to_go_remaining': 0, 'password_hash': '782af4ba40f9fbae4c081fe35a3d1220d193684e05d8776c71bbafe162d86c44', 'role': 'admin'}
direct to admin page


In [60]:
### Registration Code: Backend + Queries

test_case='correct_input'
# 730473910,Kayla,Casey,testPassword
if test_case=='correct_input':
    entered_pid='756589321'
    entered_first='john'
    entered_last='doe'
    entered_password1='passieword'
    entered_password2='passieword'
elif test_case=='pid_in_use':
    entered_pid='730330697'
    entered_first='jerry'
    entered_last='doubles'
    entered_password1='passieword'
    entered_password2='passieword'
elif test_case=='password_mismatch':
    entered_pid='756589322'
    entered_first='jack'
    entered_last='downs'
    entered_password1='passieword1'
    entered_password2='passieword'
elif test_case=='missing_field_pid':
    entered_pid=''
    entered_first='jenny'
    entered_last='dawkins'
    entered_password1='passieword2'
    entered_password2='passieword2'
elif test_case=='missing_field_first_name':
    entered_pid='756589323'
    entered_first=''
    entered_last='deere'
    entered_password1='passieword3'
    entered_password2='passieword3'
elif test_case=='missing_field_last_name':
    entered_pid='756589324'
    entered_first='janice'
    entered_last=''
    entered_password1='passieword4'
    entered_password2='passieword4'
elif test_case=='missing_field_password1':
    entered_pid='756589325'
    entered_first='james'
    entered_last='duley'
    entered_password1=''
    entered_password2='passieword5'
elif test_case=='missing_field_password2':
    entered_pid='756589326'
    entered_first='javier'
    entered_last='dalton'
    entered_password1='passieword6'
    entered_password2=''


if entered_pid=='' or not entered_pid.isdigit() or len(entered_pid) != 9: # (check 9 digit num)
    print('Enter valid PID')
elif entered_first=='' or not entered_first.isalpha(): # check not empty, only letters (no num, whitespace)
    print('Enter valid First Name')
elif entered_last=='' or not entered_last.isalpha(): # check not empty, only letters (no num, whitespace)
    print('Enter valid Last Name')
elif entered_password1=='' or len(entered_password1) > 24 or len(entered_password1) < 8:  # checks empty, add other check for validity
    print('Enter valid password; must be between 8 and 24 characters')
elif entered_password2 != entered_password1: # triggers if first valid but second doesn't match
    print('Passwords do Not Match')
else: # means all valid

    # Check if PID in use
    cursor.execute('SELECT * FROM Students S WHERE S.pid=%s', entered_pid)
    result = cursor.fetchall()
    try:
        result[0][0] # if returns a result, it's being used
        print('PID Already in Use')
    except: # may be a bad idea as any error will allow it to continue, but I think the only error for this would be an index error meaning that there's no PID in use
        new_user_details = (entered_pid, entered_first.title(), entered_last.title(), hash_password(entered_password1))
        cursor.execute('INSERT INTO Students (pid, first_name, last_name, password_hash) VALUES (%s, %s, %s, %s);', new_user_details)
        db.commit()

        cursor.execute('SELECT * FROM Students S WHERE S.pid=%s', entered_pid)
        result = cursor.fetchall()

        print('Sign-Up Success')
        current_user = {
            'pid': result[0][0],
            'first_name': result[0][1],
            'last_name': result[0][2],
            'balance': result[0][3],
            'to_go_remaining': result[0][4],
            'password_hash': result[0][5],
            'role': result[0][6],
        }
        print(current_user)

Sign-Up Success
{'pid': 756589321, 'first_name': 'John', 'last_name': 'Doe', 'balance': Decimal('0.00'), 'to_go_remaining': 2, 'password_hash': 'a5cf05e056a48636dfebb5b6f493646035606ec4d60bf5f5cec2bdf39e977f26', 'role': 'student'}


In [71]:
### Landing Page
# based on current user from sign-in cell; 

### Header 
# Common Info
# print(f'Name: {current_user["first_name"]} {current_user["last_name"]}')
print(f'Name: {current_user["first_name"]} {current_user["last_name"]}')
print(f'PID: {current_user["pid"]}')
print(f'{current_user["role"].title()} View')

# Right Header
if current_user["role"]=='student':
    print(f'Account Balance: ${current_user["balance"]}')
    print(f'To-Go Boxes Remaining: {current_user["to_go_remaining"]}/2')
if current_user["role"]=='admin':
    print('Button for New Order')
    print('Button for Student Management')

# Transaction History
if current_user["role"]=='student':
    cursor.execute('SELECT * FROM Transactions T WHERE T.pid=%s', current_user["pid"])
    result = cursor.fetchall()
    print(result)
elif current_user["role"]=='admin':
    cursor.execute('SELECT * FROM Transactions T')
    result = cursor.fetchall()
    print(result)



Name: Guy Feeyarey
PID: 987654321
Admin View
Button for New Order
Button for Student Management
((1, 730473910, 1, 'dine-in', datetime.date(2024, 11, 10)), (2, 730560194, 2, 'dine-in', datetime.date(2024, 11, 11)), (3, 730330697, 3, 'dine-in', datetime.date(2024, 11, 9)), (4, 730578592, 4, 'to-go', datetime.date(2024, 11, 10)), (5, 730330697, 2, 'to-go', datetime.date(2024, 11, 10)))


In [62]:
### Be sure to close at the end
### Note that if you close and then try to perform new queries you'll get an InterfaceError: (0, '') error and need to reopen conneciton
db.close()